In [13]:
%matplotlib inline
import pandas as pd


from IPython.display import display

In [14]:
changes = pd.read_csv("tariff_changes.csv")
changes["CHANGE_MONTH"] = pd.to_datetime(changes["CHANGE_MONTH"])
changes.head()


,SUBSCRIBER_ID,TARIFF_PLAN_ID_FROM,TARIFF_PLAN_ID_TO,CHANGE_MONTH
0,1,2,3,2017-03-01
1,2,1,5,2017-06-01
2,3,1,5,2017-04-01
3,4,3,5,2017-01-01
4,5,1,5,2017-05-01


In [15]:
changes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6377 entries, 0 to 6376
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   SUBSCRIBER_ID        6377 non-null   int64         
 1   TARIFF_PLAN_ID_FROM  6377 non-null   int64         
 2   TARIFF_PLAN_ID_TO    6377 non-null   int64         
 3   CHANGE_MONTH         6377 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 199.4 KB


In [16]:
bills = pd.read_csv("plan_change_with_bills.csv")
bills.head()


,SUBSCRIBER_ID,TARIFF_PLAN_ID_FROM,TARIFF_PLAN_ID_TO,CHANGE_MONTH,BILL_MONTH,CHARGES,VALUE
0,1,2,3,2017-03-01,2016-12-01,21.132,BILL_BEFORE
1,1,2,3,2017-03-01,2017-06-01,23.651,BILL_AFTER
2,1,2,3,2017-03-01,2017-02-01,27.858,BILL_BEFORE
3,1,2,3,2017-03-01,2017-05-01,27.487,BILL_AFTER
4,1,2,3,2017-03-01,2017-04-01,23.983,BILL_AFTER


In [17]:
bills.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38155 entries, 0 to 38154
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   SUBSCRIBER_ID        38155 non-null  int64  
 1   TARIFF_PLAN_ID_FROM  38155 non-null  int64  
 2   TARIFF_PLAN_ID_TO    38155 non-null  int64  
 3   CHANGE_MONTH         38155 non-null  object 
 4   BILL_MONTH           38155 non-null  object 
 5   CHARGES              38155 non-null  float64
 6   VALUE                38155 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 2.0+ MB


In [18]:
month_cnt = bills.groupby(["SUBSCRIBER_ID", "TARIFF_PLAN_ID_FROM", "TARIFF_PLAN_ID_TO", "VALUE"])[
    "BILL_MONTH"].agg(len).unstack("VALUE").reset_index()

month_cnt.head()


VALUE,SUBSCRIBER_ID,TARIFF_PLAN_ID_FROM,TARIFF_PLAN_ID_TO,BILL_AFTER,BILL_BEFORE
0,1,2,3,3.0,3.0
1,2,1,5,3.0,3.0
2,3,1,5,3.0,3.0
3,4,3,5,3.0,3.0
4,5,1,5,3.0,3.0


In [19]:
suspended = pd.read_csv("../data/Suspended.csv")

suspended.loc[suspended["END_DT"] == "$null$",
              "END_DT"] = "2018-01-18"  # Max END_DT + 1 day
for col in "START_DT", "END_DT":
    suspended[col] = pd.to_datetime(suspended[col])

# вычислим время отключения абонента:
suspended["DAYS_CNT"] = (suspended["END_DT"] -
                         suspended["START_DT"]).dt.days + 1

suspended.head()


,SUBSCRIBER_ID,START_DT,END_DT,STATUS,DAYS_CNT
0,5195,2017-12-27,2017-12-27,Suspended,1
1,5198,2017-04-18,2017-04-18,Suspended,1
2,381,2017-04-18,2017-04-20,Suspended,3
3,3500,2017-12-21,2018-01-18,Suspended,29
4,3302,2016-08-23,2016-09-07,Suspended,16


In [20]:
merged = changes.merge(suspended, on="SUBSCRIBER_ID", how="left")


In [21]:
from functools import reduce

# Условия для даты до `CHANGE_DTTM`
before_conditions = [
    merged["CHANGE_MONTH"].apply(
        lambda x: x + pd.DateOffset(months=-3)) < merged["END_DT"],
    merged["START_DT"] <= merged["CHANGE_MONTH"],
]
before_change_mask = reduce(lambda x, y: x & y, before_conditions)

# Условия для даты  после `CHANGE_DTTM`
after_conditions = [
    merged["CHANGE_MONTH"].apply(
        lambda x: x + pd.DateOffset(months=3)) > merged["START_DT"],
    merged["END_DT"] >= merged["CHANGE_MONTH"],
]
after_change_mask = reduce(lambda x, y: x & y, after_conditions)


merged.loc[before_change_mask, "VALUE"] = "BEFORE"
merged.loc[after_change_mask,  "VALUE"] = "AFTER"

suspended_sum = merged.groupby(["SUBSCRIBER_ID", "TARIFF_PLAN_ID_FROM", "TARIFF_PLAN_ID_TO", "VALUE"])[
    "DAYS_CNT"].sum().unstack("VALUE").reset_index()

suspended_sum.head()


VALUE,SUBSCRIBER_ID,TARIFF_PLAN_ID_FROM,TARIFF_PLAN_ID_TO,AFTER,BEFORE
0,5,1,5,29.0,24.0
1,8,3,4,143.0,70.0
2,9,3,4,145.0,NaN
3,29,1,5,NaN,25.0
4,31,5,4,6.0,NaN


In [22]:
suspended_monthly = month_cnt.merge(suspended_sum,
                                    on=["SUBSCRIBER_ID", "TARIFF_PLAN_ID_FROM", "TARIFF_PLAN_ID_TO"], how="left").fillna(0)

for part in "BEFORE", "AFTER":
    suspended_monthly[part] = \
        suspended_monthly.eval("%s / BILL_%s" % (part, part))

suspended_monthly.head()


VALUE,SUBSCRIBER_ID,TARIFF_PLAN_ID_FROM,TARIFF_PLAN_ID_TO,BILL_AFTER,BILL_BEFORE,AFTER,BEFORE
0,1,2,3,3.0,3.0,0.000000,0.0
1,2,1,5,3.0,3.0,0.000000,0.0
2,3,1,5,3.0,3.0,0.000000,0.0
3,4,3,5,3.0,3.0,0.000000,0.0
4,5,1,5,3.0,3.0,9.666667,8.0


In [23]:
suspended_total = suspended_monthly[["BEFORE", "AFTER"]].mean()
suspended_total = suspended_total["AFTER"] - suspended_total["BEFORE"]

print("Average time of disconnection of the tariff plan: %.2f days" %
      suspended_total)


Average time of disconnection of the tariff plan: 0.44 days


In [24]:
tmp = suspended_monthly.groupby(["TARIFF_PLAN_ID_FROM", "TARIFF_PLAN_ID_TO"])[
    ["AFTER", "BEFORE"]].mean().reset_index()

tmp["CHANGE"] = tmp.eval("AFTER - BEFORE")
pvt = pd.pivot_table(data=tmp, columns="TARIFF_PLAN_ID_TO", index="TARIFF_PLAN_ID_FROM", aggfunc=sum, values="CHANGE")\
    .fillna(0).round(2)

print("Average growth in blocking per subscriber per month")
display(pvt)


Average growth in blocking per subscriber per month


TARIFF_PLAN_ID_TO,1,2,3,4,5
TARIFF_PLAN_ID_FROM,,,,,
1,0.00,0.47,0.50,0.81,0.13
2,1.79,0.00,0.78,1.69,0.01
3,-0.11,1.72,0.00,1.58,0.09
4,1.84,4.35,0.65,0.00,1.21
5,0.43,0.00,2.70,0.25,0.00


                        Conclusions:
The greatest number of blockages occurs when changing the tariff plan from #4 to #2